In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:200
    scenarios[i] = DataFrame(scenarios_200[:,3*i-2:3*i])
end

In [4]:
scenarios[198]

Row,price_197,wind power_197,grid_excess_197
,Float64,Float64,Int64
1,96.41,1597.95,1
2,95.08,1399.15,1
3,93.63,1121.67,0
4,89.17,927.533,1
5,88.97,829.228,1
6,93.69,792.45,1
7,100.5,668.976,0
8,101.92,326.361,1
9,101.95,394.394,0


In [5]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [6]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:24] >= 0)
@variable(model, delta[1:200,1:24])
@variable(model, delta_up[1:200,1:24] >= 0)
@variable(model, delta_down[1:200,1:24] >= 0)

# Define the objective function
@objective(model, Min, sum( sum( 1/200 * (scenarios[i][hour,"price"] * p_DA[hour] + 0.9*scenarios[i][hour,"price"]*delta_up[i,hour] - 1.2*scenarios[i][hour,"price"]*delta_down[i,hour]) for i in 1:200) for hour in 1:24))

# Define the constraints
@constraint(model, [hour in 1:24], p_DA[hour] <= 200)
@constraint(model, [i in 1:200, hour in 1:24], delta[i,hour] = scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:200, hour in 1:24], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:200, hour in 1:24], delta_down[i,hour] >= 0)
@constraint(model, [i in 1:200, hour in 1:24], delta_up[i,hour] >= 0)

# Solve the optimization problem
optimize!(model)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


ArgumentError: ArgumentError: column name "price" not found in the data frame; existing most similar names are: "price_1"